Step 5: Test Your Refactored Code

Test with valid JSON file:

In [ ]:
"""# Create a sample products.json in the current working directory
sample_data = {
    "products": [
        {
            "id": "p1",
            "name": "Water Bottle",
            "category": "Outdoors",
            "price": 12.99,
            "features": ["BPA-free", "Insulated"]
        }
    ]
}

with open("products.json", "w", encoding="utf-8") as f:
    json.dump(sample_data, f, indent=2)"""


1 What happens if products.json doesn't exist?

In this code, with open(json_file, 'r') as f: will raise a FileNotFoundError, and the program stops because there’s no try/except around it.

2 What happens if JSON is invalid?

json.load(f) will raise a json.JSONDecodeError, and since there’s no try/except around it, the program stops with a traceback.

3 What happens if product validation fails?

Product(**item) raises a validation error, but the except: pass swallows it, so that product is silently skipped with no message.

4 What happens if API call fails?

It raises an exception (network error, auth error, rate limit, etc.) and the program stops because there’s no try/except around the API call.

5 What functions do multiple things?

The main one is generate_product_descriptions(): it does file I/O, JSON parsing, validation, prompt building, API calls, response parsing, and writing results. The validator price_must_be_positive() is single-purpose.

6 Where is code repeated that could be a helper function?

The prompt-building block inside the loop is a good candidate to extract into a helper (e.g., build_prompt(product)). If you later add more API calls or variants, the API call + response parsing could also become a helper.


Step 2: Create Helper Functions

def load_json_file(file_path: str) -> dict:
    """Load and parse JSON file with error handling."""
    # TODO: Implement with proper error handling
    # Should show WHERE error occurs if file not found or JSON invalid
    pass

def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic."""
    # TODO: Implement with proper error handling
    # Should show WHICH fields are invalid and WHY
    pass

def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    # TODO: Extract prompt creation logic
    pass

def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    # TODO: Extract response parsing logic
    pass

def format_output(product: Product, description: str) -> dict:
    """Format final output."""
    # TODO: Extract output formatting logic
    pass


1- Parsing of the json file (helper function N1)

In [9]:
import json
import os

def load_json_file(file_path: str) -> dict:
    """Load and parse JSON file with error handling."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        error_msg = (
            "ERROR in load_json_file(): FileNotFoundError\n"
            f"  Location: File '{file_path}' not found\n"
            f"  Current directory: {os.getcwd()}\n"
            "  Suggestion: Check that the file path/location is correct"
        )
        print(error_msg)
        raise
    except json.JSONDecodeError as e:
        error_msg = (
            "ERROR in load_json_file(): JSONDecodeError\n"
            f"  Location: File '{file_path}', line {e.lineno}, column {e.colno}\n"
            f"  Reason: {e.msg}\n"
            f"  Suggestion: Check JSON syntax at line {e.lineno}"
        )
        print(error_msg)
        raise



File validation (helper function N2)

In [13]:
from typing import Optional, List
from pydantic import BaseModel, Field, ValidationError

# 1. Define your schema with constraints
class Product(BaseModel):
    id: int
    name: str = Field(..., min_length=2)
    price: float = Field(..., gt=0)
    category: Optional[str] = "General"
    features: List[str] = []

# 2. The Validation Function
def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic."""
    try:
        # Dictionary unpacking: tries to map dict keys to Product fields
        return Product(**product_dict)
    
    except ValidationError as e:
        # Build a highly readable error report
        error_msg = (
            "ERROR in validate_product_data(): ValidationError\n"
            f"  Product ID: {product_dict.get('id', 'unknown')}\n"
            "  Invalid fields:\n"
        )
        
        # Pydantic's .errors() returns a list of dictionaries with error details
        for error in e.errors():
            # 'loc' is a tuple (e.g., ('price',)) representing the path to the field
            field = ".".join(str(x) for x in error["loc"])
            # 'msg' contains the human-readable reason (e.g., 'less than 0')
            error_msg += f"    - {field}: {error['msg']}\n"
            
        error_msg += "  Suggestion: Fix the invalid fields above"
        print(error_msg)
        return None


Prompt creation (helper function N3)

In [ ]:
def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    return f"""Create a product description for:
Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Generate a compelling product description."""


Parse the api response (helper function N4)

In [ ]:
def parse_api_response(response) -> str:
    """Parse OpenAI API response with detailed error reporting."""
    try:
        # Check if response exists
        if response is None:
            raise ValueError("Response is None")
        
        # Check for choices
        if not hasattr(response, 'choices'):
            raise ValueError(f"Response missing 'choices'. Got type: {type(response).__name__}, keys: {getattr(response, '__dict__', {}).keys()}")
        
        if not response.choices:
            raise ValueError("Response 'choices' is empty")
        
        # Check for message
        choice = response.choices[0]
        if not hasattr(choice, 'message'):
            raise ValueError(f"Choice missing 'message'. Got: {type(choice).__name__}")
        
        # Check for content
        if not hasattr(choice.message, 'content'):
            raise ValueError(f"Message missing 'content'. Got: {type(choice.message).__name__}")
        
        if choice.message.content is None:
            raise ValueError("Message content is None (possibly a function call or refusal)")
        
        return choice.message.content
    
    except ValueError:
        raise
    except Exception as e:
        error_msg = (
            "ERROR in parse_api_response(): Unexpected structure\n"
            f"  Error type: {type(e).__name__}\n"
            f"  Error message: {str(e)}\n"
            f"  Response type: {type(response).__name__}\n"
            f"  Response preview: {str(response)[:200]}"
        )
        print(error_msg)
        raise ValueError(error_msg)

Format Final output (helper function N5)

In [10]:
def format_output(product: Product, description: str) -> dict:
    """Format final output with error handling."""
    try:
        if product is None:
            raise ValueError("Product is None")
        if not isinstance(description, str):
            raise ValueError(f"Description must be str, got {type(description).__name__}")
        
        return {
            "product_id": product.id,
            "name": product.name,
            "description": description,
        }
    except AttributeError as e:
        error_msg = (
            "ERROR in format_output(): AttributeError\n"
            f"  Product type: {type(product).__name__}\n"
            f"  Missing attribute: {str(e)}\n"
            "  Suggestion: Ensure product is a valid Product instance"
        )
        print(error_msg)
        raise


Step 3: Modularize Functions

In [12]:
"""def load_and_validate_products(json_path: str) -> List[Product]:
    "Load JSON and validate products."
    # Uses load_json_file() and validate_product_data()
    # Should show WHERE errors occur
    pass

def generate_description(product: Product, api_client) -> str:
    "Generate description for one product using API."
    # Uses create_product_prompt() and parse_api_response()
    # Should show WHERE API errors occur
    pass

def process_products(products: List[Product], api_client) -> List[dict]:
    "Process all products and generate descriptions."
    # Orchestrates the processing
    # Should handle errors for each product
    pass

def save_results(results: List[dict], output_path: str) -> None:
    "Save results to JSON file."
    # Should show WHERE file errors occur
    pass"""


'def load_and_validate_products(json_path: str) -> List[Product]:\n    "Load JSON and validate products."\n    # Uses load_json_file() and validate_product_data()\n    # Should show WHERE errors occur\n    pass\n\ndef generate_description(product: Product, api_client) -> str:\n    "Generate description for one product using API."\n    # Uses create_product_prompt() and parse_api_response()\n    # Should show WHERE API errors occur\n    pass\n\ndef process_products(products: List[Product], api_client) -> List[dict]:\n    "Process all products and generate descriptions."\n    # Orchestrates the processing\n    # Should handle errors for each product\n    pass\n\ndef save_results(results: List[dict], output_path: str) -> None:\n    "Save results to JSON file."\n    # Should show WHERE file errors occur\n    pass'

1st Function

In [13]:
def load_products_json(json_path: str) -> dict:
    """Load JSON file only."""
    return load_json_file(json_path)

2nd Function

In [14]:
def validate_products_list(data: dict, source: str) -> List[Product]:
    """Validate products list with detailed error handling."""
    if data is None:
        error_msg = (
            "ERROR in validate_products_list(): Data is None\n"
            f"  Source: {source}\n"
            "  Suggestion: Check that JSON was loaded correctly"
        )
        print(error_msg)
        raise ValueError(error_msg)
    
    if not isinstance(data, dict):
        error_msg = (
            "ERROR in validate_products_list(): Invalid data type\n"
            f"  Source: {source}\n"
            f"  Expected: dict, Got: {type(data).__name__}\n"
            "  Suggestion: Ensure JSON file contains a valid object"
        )
        print(error_msg)
        raise TypeError(error_msg)
    
    if "products" not in data:
        error_msg = (
            "ERROR in validate_products_list(): Missing 'products' key\n"
            f"  Source: {source}\n"
            f"  Available keys: {list(data.keys())}\n"
            "  Suggestion: Ensure JSON has a 'products' array"
        )
        print(error_msg)
        raise KeyError(error_msg)
    
    products = []
    errors = []
    
    for idx, item in enumerate(data.get("products", [])):
        result = validate_product_data(item)
        if result is None:
            errors.append(f"Product at index {idx}")
        else:
            products.append(result)
    
    if errors:
        error_msg = (
            "ERROR in validate_products_list(): Some products failed validation\n"
            f"  Source: {source}\n"
            f"  Failed products: {', '.join(errors)}\n"
            f"  Valid products: {len(products)}/{len(data.get('products', []))}"
        )
        print(error_msg)
    
    return products


Create product prompt

In [15]:
def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    return f"""Create a product description for:
Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Generate a compelling product description."""


NameError: name 'Product' is not defined

Parse api response

In [16]:
def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    try:
        return response.choices[0].message.content
    except (AttributeError, IndexError, KeyError, TypeError):
        raise ValueError("Unexpected API response structure; missing choices/message/content")


Process product

In [17]:
def process_single_product(product: Product, api_client) -> dict:
    """Process one product and return formatted output."""
    description = generate_description(product, api_client)
    return format_output(product, description)


NameError: name 'Product' is not defined

Process all products

In [18]:
def process_products(products: List[Product], api_client) -> List[dict]:
    """Process all products and generate descriptions."""
    results = []
    for product in products:
        try:
            results.append(process_single_product(product, api_client))
        except Exception as e:
            # Keep going but surface which product failed
            results.append({
                "product_id": product.id,
                "name": product.name,
                "error": str(e),
            })
    return results


NameError: name 'List' is not defined

Save results to json file

In [19]:
def save_results(results: List[dict], output_path: str) -> None:
    """Save results to JSON file."""
    try:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2)
    except OSError as e:
        raise OSError(f"Failed to write results to {output_path}: {e}")


NameError: name 'List' is not defined

Step 4: Add Error Handling (CRITICAL)

In [20]:
import json
import os

def load_json_file(file_path: str) -> dict:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        error_msg = (
            "ERROR in load_json_file(): FileNotFoundError\n"
            f"  Location: File '{file_path}' not found\n"
            f"  Current directory: {os.getcwd()}\n"
            "  Suggestion: Check that the file path/location is correct"
        )
        print(error_msg)
        raise
    except json.JSONDecodeError as e:
        error_msg = (
            "ERROR in load_json_file(): JSONDecodeError\n"
            f"  Location: File '{file_path}', line {e.lineno}, column {e.colno}\n"
            f"  Reason: {e.msg}"
        )
        print(error_msg)
        raise


JSONDecodeError: Show line number and character position

In [21]:
def load_json_file(file_path: str) -> dict:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        ...
    except json.JSONDecodeError as e:
        ...
        error_msg = (
            "ERROR in load_json_file(): JSONDecodeError\n"
            f"  Location: File '{file_path}', line {e.lineno}, column {e.colno}\n"
            f"  Reason: {e.msg}\n"
            f"  Suggestion: Check JSON syntax at line {e.lineno}"
        )                                                   
        

Pydantic ValidationError: Show which fields are invalid and why

In [22]:
from pydantic import ValidationError

def validate_product_data(product_dict: dict) -> Optional[Product]:
    try:
        return Product(**product_dict)
    except ValidationError as e:
        error_msg = (
            "ERROR in validate_product_data(): ValidationError\n"
            f"  Product ID: {product_dict.get('id', 'unknown')}\n"
            "  Invalid fields:\n"
        )
        for error in e.errors():
            field = ".".join(str(x) for x in error["loc"])
            error_msg += f"    - {field}: {error['msg']}\n"
        error_msg += "  Suggestion: Fix the invalid fields above"
        print(error_msg)
        return None


NameError: name 'Product' is not defined

OpenAI API errors: Show error type, status code, and message

In [24]:
from openai import APIError

def generate_description(product: Product, api_client) -> str:
    try:
        response = api_client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": create_product_prompt(product)}],
        )
        return parse_api_response(response)
    except APIError as e:
        error_msg = (
            "ERROR in generate_description(): APIError\n"
            f"  Product: {product.name} (ID: {product.id})\n"
            f"  Error type: {type(e).__name__}\n"
            f"  Status code: {getattr(e, 'status_code', 'N/A')}\n"
            f"  Message: {str(e)}\n"
            "  Suggestion: Check API key, rate limits, or try again later"
        )
        print(error_msg)
        raise


NameError: name 'Product' is not defined

Network errors: Show timeout/connection details

In [ ]:
import requests

def handle_network_error(function_name: str, context: str, e: Exception) -> None:
    error_type = type(e).__name__
    error_message = str(e)
    if isinstance(e, requests.Timeout):
        error_message = f"Timeout while connecting: {e}"
    elif isinstance(e, requests.ConnectionError):
        error_message = f"Connection failed: {e}"

    error_msg = (
        f"ERROR in {function_name}(): {error_type}\n"
        f"  Location: {context}\n"
        f"  Message: {error_message}\n"
        "  Suggestion: Check internet connection, VPN/proxy, or try again later"
    )
    print(error_msg)
    raise



It detects requests.Timeout and requests.ConnectionError and replaces the message with more specific text, so you see timeout vs connection failure details in the output.